In [2]:
from sketch.examples.prompt_machine import *

async def chatbot(response_callback):
    messages = []
    async with websockets.connect(PM_SETTINGS['uri'], extra_headers={"Cookie": f"Authorization=Bearer {PM_SETTINGS['BOT_TOKEN']}"}) as ws:
        while True:
            try:
                raw_data = await asyncio.wait_for(ws.recv(), timeout=0.5)
                data = json.loads(raw_data)
                print("log:", data['message'])
                if data.get("replay", False) or data["sender"] == "datAI":
                    messages.append(data)
                    continue
                if data["message"] == "break":
                    break
                if data.get("meta", False):
                    continue
                messages.append(data)
            except asyncio.TimeoutError:
                # not sure why I'm doing this, it felt important to have an "infinite" loop...
                continue
            response = response_callback("\n".join([f"{m['sender']}: {m['message']}" for m in messages]))
            await ws.send(json.dumps({"message": response}))


In [4]:
# await response_callback